In [1]:
#import Label
#import sen12ms_dataLoader as loader
import rasterio
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline
from sklearn.cluster import KMeans

In [4]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout,AdaptiveAvgPool2d
from torch.optim import Adam, SGD
import torch.nn.functional as F
from torchvision import models

import time
from sklearn.metrics import accuracy_score

In [5]:
metadata = pd.read_csv("/Volumes/ksankaran/sen12ms_subsample2/metadata_labels.csv")

In [6]:
kmeans=KMeans(n_clusters=5, random_state=0).fit(metadata[['longitude','latitude']])
metadata['clusters']=kmeans.labels_

In [7]:
metadata.head()

,Unnamed: 0,Unnamed: 0.1,image_basename,label_basename,roi,season,scene,patch,image_path,label_path,longitude,latitude,id,label_id,clusters
0,0,1,ROIs1970_fall_s2_21_p694.tif,ROIs1970_fall_lc_21_p694.tif,ROIs1970,fall,21,p694,/Volumes/ksankaran/Sen12MS/ROIs1970_fall/s2_21...,/Volumes/ksankaran/Sen12MS/ROIs1970_fall/lc_21...,-2.08365,35.050345,131,4,3
1,1,2,ROIs1970_fall_s2_21_p724.tif,ROIs1970_fall_lc_21_p724.tif,ROIs1970,fall,21,p724,/Volumes/ksankaran/Sen12MS/ROIs1970_fall/s2_21...,/Volumes/ksankaran/Sen12MS/ROIs1970_fall/lc_21...,-2.08365,35.050345,131,4,3
2,2,3,ROIs1970_fall_s2_21_p597.tif,ROIs1970_fall_lc_21_p597.tif,ROIs1970,fall,21,p597,/Volumes/ksankaran/Sen12MS/ROIs1970_fall/s2_21...,/Volumes/ksankaran/Sen12MS/ROIs1970_fall/lc_21...,-2.08365,35.050345,131,6,3
3,3,4,ROIs1970_fall_s2_21_p244.tif,ROIs1970_fall_lc_21_p244.tif,ROIs1970,fall,21,p244,/Volumes/ksankaran/Sen12MS/ROIs1970_fall/s2_21...,/Volumes/ksankaran/Sen12MS/ROIs1970_fall/lc_21...,-2.08365,35.050345,131,10,3
4,4,5,ROIs1970_fall_s2_21_p719.tif,ROIs1970_fall_lc_21_p719.tif,ROIs1970,fall,21,p719,/Volumes/ksankaran/Sen12MS/ROIs1970_fall/s2_21...,/Volumes/ksankaran/Sen12MS/ROIs1970_fall/lc_21...,-2.08365,35.050345,131,6,3


In [8]:
X = np.load('X.npy')
y = np.load('y.npy')

In [9]:
X.shape,y.shape

((3000, 13, 256, 256), (3000,))

In [10]:
def split_data(metadata,X,y,source_cluster,test_size=0.3,random_state = 1234):
    X_source = X[metadata.clusters == source_cluster]
    X_source = np.array(X_source)
    y_source = y[metadata.clusters == source_cluster]
    y_source = np.array(y_source)
    
    X_target = X[metadata.clusters != source_cluster]
    X_target = np.array(X_target)
    y_target = y[metadata.clusters != source_cluster]
    y_target = np.array(y_target)
    
    X_target_train, X_target_test, y_target_train, y_target_test = train_test_split(X_target ,
                                                                                y_target, 
                                                                                test_size = test_size, 
                                                                                #stratify=y_target,
                                                                                random_state = random_state)
    return X_source, y_source, X_target_train, X_target_test,  y_target_train, y_target_test, X_target,y_target

In [11]:
X_source, y_source, X_target_train, X_target_test,  y_target_train, y_target_test, X_target,y_target = split_data(metadata,X,y,source_cluster = 0,test_size = 0.01)

In [12]:
#np.save('X_target_test',X_target_test)
#np.save('y_target_test',y_target_test)
#X_target_test = np.load('X_target_test.npy')
#y_target_test = np.load('y_target_test.npy')

In [13]:
(X_source.shape,y_source.shape,X_target_train.shape,X_target_test.shape)

((300, 13, 256, 256), (300,), (2673, 13, 256, 256), (27, 13, 256, 256))

In [28]:
#np.save('/Volumes/ksankaran/sen12ms_subsample2/X_source',X_source)
#np.save('/Volumes/ksankaran/sen12ms_subsample2/y_source',y_source)

In [14]:
y_source

array([3, 3, 3, 3, 3, 3, 3, 3, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5,
       3, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 3, 3, 3, 3, 3, 5,
       3, 3, 3, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 5, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 3, 3, 3, 5, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 3, 3, 2, 4, 2, 2,
       2, 2, 2, 2, 4, 2, 2, 5, 2, 2, 2, 2, 4, 2, 2, 2, 2, 4, 2, 2, 2, 2,
       4, 2, 2, 2, 2, 4, 4, 2, 2, 2, 4, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 4,
       2, 2, 2, 4, 2, 2, 2, 2, 2, 4, 2, 2, 4, 2, 2, 4, 4, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 4,
       2, 9, 4, 2, 2, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 2, 2,
       2, 2, 4, 4, 2, 2, 2, 4, 2, 2, 2, 9, 2, 2, 2, 4, 2, 2, 2, 2, 2, 4,
       2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2])

In [15]:
y_source_copy = np.copy(y_source)
y_target_train = np.copy(y_target_train)
y_target_test = np.copy(y_target_test)

In [16]:
y_source = y_source - y_source.min()

In [17]:
X_source = X_source.astype(float)
X_target_train = X_target_train.astype(float)
X_target_test = X_target_test.astype(float)

In [18]:
# converting training images into torch format
X_source = torch.from_numpy(X_source)
X_target_train = torch.from_numpy(X_target_train)
X_target_test = torch.from_numpy(X_target_test)
X_target_train.shape, X_target_test.shape

(torch.Size([2673, 13, 256, 256]), torch.Size([27, 13, 256, 256]))

In [19]:
y_source = torch.from_numpy(y_source)
y_target_train = torch.from_numpy(y_target_train)
y_target_test = torch.from_numpy(y_target_test)

In [15]:
#######################

In [26]:
class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(13, 64, kernel_size=(5,5), stride=1, padding=2),
            BatchNorm2d(64),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=(3,3), stride=(2,2),padding = 1),
            
            Conv2d(64, 128, kernel_size=(5,5), stride=1, padding=2),
            BatchNorm2d(128),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=(3,3), stride=2, padding = 1),
            
            Conv2d(128, 256, kernel_size=(5,5), stride=1, padding=2),
            BatchNorm2d(256),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=(3,3), stride=2, padding = 1),
            
            Conv2d(256, 512, kernel_size=(5,5), stride=1, padding=2),
            BatchNorm2d(512),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=(3,3), stride=2, padding = 1),
            
            AdaptiveAvgPool2d((1, 1))
        )
        self.linear_layers = Sequential(
            #AdaptiveAvgPool2d(output_size=(1, 1)),
            Linear(in_features= 512, out_features=8, bias = True)
        )
        
        #self.linear = Sequential(
         #   AdaptiveAvgPool2d(output_size=(1, 1)),
        #    Linear(in_features=512, out_features=10, bias=True)
        #)


    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x



In [27]:
# defining the model
model = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.0001)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

print(model)

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(13, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=1, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=(3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=(3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
    (12): Conv2d(256, 512, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (13): BatchNorm2d(512, eps=1e-

In [28]:
model = model.double()

In [29]:
X_target_test.size()

torch.Size([27, 13, 256, 256])

In [30]:
# batch size of the model
batch_size = 8

# number of epochs to train the model
n_epochs = 5

for epoch in range(1, n_epochs+1):
    print("epoch",epoch,'\n')

    # keep track of training and validation loss
    train_loss = 0.0
        
    #permutation = torch.randperm(X_source.size()[0])
    permutation = torch.randperm(X_source.size()[0])

    training_loss = []
    #for i in tqdm(range(0,X_source.size()[0], batch_size)):
    for i in tqdm(range(0,X_source.size()[0], batch_size)):
        print("i=",i,'\n')

        print("1")
        indices = permutation[i:i+batch_size]
        #batch_x, batch_y = X_source[indices],y_source[indices]
        batch_x, batch_y = X_source[indices],y_source[indices]
        print("2")
        if torch.cuda.is_available():
            batch_x, batch_y = batch_x.cuda(), batch_y.cuda()
        
        optimizer.zero_grad()
        print("3")
        outputs = model(batch_x)
        outputs.squeeze(1)
        print(outputs.size())
        print(batch_y.size())
        loss = criterion(outputs,batch_y)

        print("4")
        training_loss.append(loss.item())
        loss.backward()
        optimizer.step()
        
    training_loss = np.average(training_loss)
    print('epoch: \t', epoch, '\t training loss: \t', training_loss)

  0%|          | 0/38 [00:00<?, ?it/s]

epoch 1 

i= 0 

1
2
3
torch.Size([8, 8])
torch.Size([8])
4


  3%|▎         | 1/38 [00:45<27:48, 45.10s/it]

i= 8 

1
2
3
torch.Size([8, 8])
torch.Size([8])
4


  5%|▌         | 2/38 [01:22<25:39, 42.77s/it]

...

i= 288 

1
2
3
torch.Size([8, 8])
torch.Size([8])
4


 97%|█████████▋| 37/38 [24:53<00:40, 40.12s/it]

i= 296 

1
2
3
torch.Size([4, 8])
torch.Size([4])
4


100%|██████████| 38/38 [25:07<00:00, 39.68s/it]


epoch: 	 5 	 training loss: 	 0.46500585118407567
